### To-Do: Figure out why this isn't giving at least 10 recommendations

In [109]:
import pandas as pd
import numpy as np
from copy import copy
from surprise import Dataset
from surprise import Reader
from surprise import KNNWithMeans, SVD
from surprise.model_selection import GridSearchCV
from surprise.model_selection import train_test_split
from surprise import accuracy
from collections import defaultdict

#### We want to find users with similar taste in books and try to predict what we will like based on what they like ####

##### step 1: what do I like? #####

In [110]:
# Execute content-based filtering notebook to created needed data:

In [111]:
!jupyter nbconvert --to notebook --execute --inplace --no-input test_content_rec_Susanne_Lay.ipynb

[NbConvertApp] Converting notebook test_content_rec_Susanne_Lay.ipynb to notebook
[NbConvertApp] Writing 304639 bytes to test_content_rec_Susanne_Lay.ipynb


In [112]:
data = pd.read_csv('data/books_ratings.csv')
data.head()

,isbn,book_title,book_author,year_of_publication,publisher,genre,user_id,book_rating
0,068160204X,The Royals,Kitty Kelley,2020,Bausch & Lombard,NaN,16634,0
1,068160204X,The Royals,Kitty Kelley,2020,Bausch & Lombard,NaN,87141,0
2,068160204X,The Royals,Kitty Kelley,2020,Bausch & Lombard,NaN,169736,7
3,068160204X,The Royals,Kitty Kelley,2020,Bausch & Lombard,NaN,208406,0
4,068160204X,The Royals,Kitty Kelley,2020,Bausch & Lombard,NaN,230496,0


In [113]:
average_rating = pd.read_csv('data/averageRatingdf.csv')
average_rating.head()

,book_title,book_author,isbn,rating_count,average_rating,mod_titles
0,!%@ (A Nutshell handbook),Donnalyn Frey,1565920317,1,6.0,A Nutshell handbook
1,!%@ (A Nutshell handbook),Donnalyn Frey,1565920465,1,0.0,A Nutshell handbook
2,$30 Film School,Michael W. Dean,1592000673,1,8.0,30 Film School
3,$oft Money: The True Power in Our Nation's Cap...,E. L. Burton,1588204030,5,5.4,oft Money The True Power in Our Nation s Cap...
4,' Mein verwundetes Herz'. Das Leben der Lilli ...,Martin Doerry,342105634X,1,9.0,Mein verwundetes Herz Das Leben der Lilli ...


In [114]:
data['user_id'] = data['user_id'].astype('str') 

In [115]:
# filtering user_id who read and rated more than 15 books
user_counts = data['user_id'].value_counts()
data_users = data[data['user_id'].isin(user_counts[user_counts >= 15].index)]
data_users.sample(10)

,isbn,book_title,book_author,year_of_publication,publisher,genre,user_id,book_rating
155491,067172083X,BARRAYAR : BARRAYAR (Fireside Sports Classic),Lois McMaster Bujold,1991,Baen,"Hugo Award Winner, award:hugo_award=1992, awar...",178199,9
153363,030700144X,The Taxi That Hurried (Little Golden Book),Lucy Mitchell,1992,Golden Books,"Fiction, Taxicabs, Traffic congestion, Vehicles",153662,9
63213,1842322869,Requiem for a Wren,Nevil Shute,2000,House of Stratus,"Biography, Fiction, World War, 1939-1945, Soci...",244994,0
9492,031298376X,The Chase,Brenda Joyce,2003,St. Martin's Paperbacks,"Women detectives, Serial murders, Fiction, Wor...",226545,0
131014,1567900062,The Peacock Princess: The True-Life Story of a...,Sara Harris,1995,Cool Hand Communications Inc.,"Intermarriage, Wife abuse, Biography, Women",24945,0
55796,1586601350,The Sewing Circle: One Woman's Mentoring Shape...,Sally Laity,2001,Barbour Publishing,"American Christian fiction, American Love stor...",190925,7
141873,002026478X,AGE OF INNOCENCE (MOVIE TIE-IN),Edith Wharton,1993,Scribner,"Fiction, Triangles (Interpersonal relations), ...",113519,0
178037,2253007102,Les Fleurs Du Mal,C. Baudelaire,1972,Livre De Poche French,"Traducciones al español, Translations into Spa...",52203,8
162753,059042419X,Bsc Ss Babysitters Summer Vacation (Baby-Sitte...,Ann Martin,1989,Scholastic Inc,"Babysitters, Fiction, Baby sitters, Children's...",212898,0
124328,3404125010,Die Pfeiler der Macht.,Ken Follett,1996,LÃ?Â¼bbe,"Fiction, History, Great Britain in fiction, Up...",245499,7


In [116]:
# creating list of books for one reference user

books_read = data[data['user_id'] == '114368']
books_read 

,isbn,book_title,book_author,year_of_publication,publisher,genre,user_id,book_rating
221,042517140X,Loyalty in Death,J. D. Robb,2004,Berkley Publishing Group,"Fiction, Eve Dallas (Fictitious character), Po...",114368,5
2034,037379133X,Forbidden (Harlequin Blaze),Tori Carrington,2004,Harlequin,"Fiction, Passion, Romance, Large type books, C...",114368,0
2161,037383599X,Simply Sensual (Harlequin Single Title),Carly Phillips,2004,Harlequin,"Fiction, Bodyguards, Children of the rich in f...",114368,5
4208,1551667401,Out Of The Dark,Sharon Sala,2003,Mira,"Romance, Cults, Missing children, Ex-cultists,...",114368,5
4273,1551667436,Strange Bedpersons,Jennifer Crusie,2003,Mira,"Social action, Feminists, Conservatism, Ambiti...",114368,0
...,...,...,...,...,...,...,...,...
166713,044011585X,Crossings,DANIELLE STEEL,1987,Dell,"Fiction, World War, 1939-1945, Ocean travel, L...",114368,10
167136,037308529X,"Temptation (Silhouette Romance, No 529)",Nora Roberts,1987,Silhouette,"Open Library Staff Picks, Fiction, Camps for g...",114368,5
168482,082171838X,September Moon,Constance O'Banyon,1986,Zebra Books,NaN,114368,5
170307,037309230X,Sarah's Child (Silhouette Special Edition # 230),Linda Howard,1985,Silhouette,"Romance, Contemporary Romance, Fiction, Infant...",114368,5


In [117]:
# find similar users

In [118]:
# create a set of all the unique books that user has read
# create a set of all the unique books that user has read, including user_id

unique_books = set(books_read['isbn'])

In [119]:
len(unique_books)

100

In [120]:
# store any user that read the same book as us in the overlap_users dictionary
overlap_users = {}

# overlap_users is a dictionary with key = user_id and value is count of times that reader read a book that we also read  
for book in unique_books:
    for user in data[data['isbn'] == book]['user_id']:
        if user != '114368':
            if user in overlap_users:
                overlap_users[user] += 1
            else:
                overlap_users[user] = 1


In [121]:
len(overlap_users)  

1567

In [122]:
overlap_users

{'277945': 2,
 '254': 2,
 '2437': 1,
 '5868': 1,
 '8284': 5,
 '9354': 1,
 '13552': 14,
 '20201': 4,
 '28360': 1,
 '40889': 8,
 '43806': 3,
 '48494': 6,
 '51350': 3,
 '51365': 1,
 '53664': 1,
 '55492': 15,
 '64436': 4,
 '68568': 2,
 '74493': 1,
 '76352': 19,
 '78553': 2,
 '86641': 1,
 '95903': 1,
 '101209': 3,
 '102967': 9,
 '104211': 2,
 '111847': 2,
 '114200': 1,
 '114413': 1,
 '116599': 10,
 '123883': 15,
 '124066': 1,
 '125774': 6,
 '126903': 2,
 '129084': 3,
 '129358': 32,
 '130438': 1,
 '130474': 10,
 '134029': 1,
 '135149': 6,
 '141127': 1,
 '143253': 7,
 '145451': 16,
 '145752': 1,
 '149071': 1,
 '151098': 3,
 '152770': 2,
 '153662': 29,
 '153718': 4,
 '154469': 3,
 '155444': 2,
 '159376': 3,
 '159822': 1,
 '161752': 1,
 '162311': 2,
 '162639': 5,
 '166123': 6,
 '170518': 4,
 '172030': 7,
 '177374': 9,
 '178181': 2,
 '178240': 1,
 '181796': 2,
 '182085': 10,
 '184424': 2,
 '185233': 12,
 '185254': 3,
 '187517': 2,
 '188583': 2,
 '194669': 3,
 '194676': 3,
 '198711': 11,
 '201641

In [123]:
print(overlap_users)

{'277945': 2, '254': 2, '2437': 1, '5868': 1, '8284': 5, '9354': 1, '13552': 14, '20201': 4, '28360': 1, '40889': 8, '43806': 3, '48494': 6, '51350': 3, '51365': 1, '53664': 1, '55492': 15, '64436': 4, '68568': 2, '74493': 1, '76352': 19, '78553': 2, '86641': 1, '95903': 1, '101209': 3, '102967': 9, '104211': 2, '111847': 2, '114200': 1, '114413': 1, '116599': 10, '123883': 15, '124066': 1, '125774': 6, '126903': 2, '129084': 3, '129358': 32, '130438': 1, '130474': 10, '134029': 1, '135149': 6, '141127': 1, '143253': 7, '145451': 16, '145752': 1, '149071': 1, '151098': 3, '152770': 2, '153662': 29, '153718': 4, '154469': 3, '155444': 2, '159376': 3, '159822': 1, '161752': 1, '162311': 2, '162639': 5, '166123': 6, '170518': 4, '172030': 7, '177374': 9, '178181': 2, '178240': 1, '181796': 2, '182085': 10, '184424': 2, '185233': 12, '185254': 3, '187517': 2, '188583': 2, '194669': 3, '194676': 3, '198711': 11, '201641': 4, '206074': 4, '207349': 10, '208147': 4, '208605': 1, '212898': 8, 

In [124]:
unique_books

{'006104122X',
 '006108445X',
 '006440031X',
 '034538475X',
 '037307445X',
 '037307607X',
 '037307817X',
 '037308529X',
 '037309230X',
 '037322365X',
 '037325928X',
 '037348240X',
 '037348285X',
 '037348397X',
 '037348464X',
 '037351011X',
 '037369105X',
 '037379018X',
 '037379021X',
 '037379035X',
 '037379049X',
 '037379097X',
 '037379102X',
 '037379116X',
 '037379133X',
 '037383568X',
 '037383599X',
 '038076654X',
 '038079456X',
 '038531292X',
 '038531437X',
 '038531695X',
 '038542471X',
 '038550120X',
 '042517140X',
 '042518630X',
 '044011585X',
 '044652767X',
 '044900256X',
 '050552354X',
 '050552421X',
 '051511779X',
 '051512317X',
 '051512608X',
 '051512947X',
 '051513287X',
 '051513628X',
 '055328990X',
 '055329783X',
 '055344557X',
 '055356045X',
 '055358457X',
 '067164257X',
 '067168289X',
 '067172858X',
 '067179938X',
 '067942573X',
 '073943053X',
 '073943232X',
 '080411952X',
 '082171838X',
 '082171922X',
 '082177512X',
 '084394563X',
 '084394952X',
 '084395048X',
 '15516600

In [125]:
import pandas as pd

# Ensure books_read is a DataFrame
if not isinstance(unique_books, pd.DataFrame):
    unique_books = pd.DataFrame(unique_books, columns=["isbn"])

# Print overlap_users dictionary
print("overlap_users:", overlap_users)

# Print books_read shape
print("books_read.shape:", books_read.shape)

# Print intermediate results
for k in overlap_users:
    print(f"User: {k}, Count: {overlap_users[k]}, Threshold: {unique_books.shape[0] / 5}")

# Filter users who have read at least 20% of the same books
filtered_overlap_users = {k for k in overlap_users if overlap_users[k] > unique_books.shape[0] / 5}

# Print filtered_overlap_users
print("filtered_overlap_users:", filtered_overlap_users)

overlap_users: {'277945': 2, '254': 2, '2437': 1, '5868': 1, '8284': 5, '9354': 1, '13552': 14, '20201': 4, '28360': 1, '40889': 8, '43806': 3, '48494': 6, '51350': 3, '51365': 1, '53664': 1, '55492': 15, '64436': 4, '68568': 2, '74493': 1, '76352': 19, '78553': 2, '86641': 1, '95903': 1, '101209': 3, '102967': 9, '104211': 2, '111847': 2, '114200': 1, '114413': 1, '116599': 10, '123883': 15, '124066': 1, '125774': 6, '126903': 2, '129084': 3, '129358': 32, '130438': 1, '130474': 10, '134029': 1, '135149': 6, '141127': 1, '143253': 7, '145451': 16, '145752': 1, '149071': 1, '151098': 3, '152770': 2, '153662': 29, '153718': 4, '154469': 3, '155444': 2, '159376': 3, '159822': 1, '161752': 1, '162311': 2, '162639': 5, '166123': 6, '170518': 4, '172030': 7, '177374': 9, '178181': 2, '178240': 1, '181796': 2, '182085': 10, '184424': 2, '185233': 12, '185254': 3, '187517': 2, '188583': 2, '194669': 3, '194676': 3, '198711': 11, '201641': 4, '206074': 4, '207349': 10, '208147': 4, '208605': 1

In [126]:
filtered_overlap_users  

{'11676', '129358', '153662', '69697'}

In [127]:
# get the isbn and ratings of the books that these users have read

# Create a dataframe that contains all user_ids that are in filtered_overlap_users and add isbn and book_rating
filtered_overlap_data = data[data['user_id'].isin(filtered_overlap_users)][['user_id', 'isbn', 'book_rating']]
filtered_overlap_data

,user_id,isbn,book_rating
9,153662,1593100175,0
10,11676,1881273156,8
123,153662,1583145397,0
141,129358,084395289X,0
208,11676,042517140X,0
...,...,...,...
183298,11676,039480001X,10
183451,153662,030702153X,9
183454,11676,006440031X,7
183566,11676,044077456X,10


### For collaborative filtering: create a user / book matrix ###

In [128]:
unique_books

,isbn
0,051512317X
1,155166416X
2,1551666618
3,1551660598
4,082177512X
...,...
95,067164257X
96,1551668947
97,037379035X
98,1551666189


In [129]:
# every row if the matrix will be a different user and every column of the matrix will be a different book containing teh rating of that user for that book

# first we need to concatenate the ratings of the picked user (id = 114368) with the filtered_overlap_data

# Create a DataFrame with the ratings of the picked user

# Filter the original DataFrame by user_id
filtered_data = data[data['user_id'] == '114368']

# Merge unique_books with the filtered data to add 'book_rating' and 'user_id' columns
unique_books = unique_books.merge(filtered_data[['isbn','user_id', 'book_rating', 'book_title']], on='isbn', how='left')

unique_books

,isbn,user_id,book_rating,book_title
0,051512317X,114368,0,Rising Tides
1,155166416X,114368,5,Sweet Baby
2,1551666618,114368,0,Impetuous Innocent
3,1551660598,114368,10,Without A Trace
4,082177512X,114368,5,Say No to Joe?
...,...,...,...,...
95,067164257X,114368,5,GARDEN OF SHADOWS (Dollanger Saga (Paperback))
96,1551668947,114368,5,White Mountain
97,037379035X,114368,0,"BODY CONTACT (Blaze, 31)"
98,1551666189,114368,5,Manhunting (Mira)


In [130]:
# concat the unique_books and filtered_overlap_data
filtered_overlap_data = pd.concat([unique_books, filtered_overlap_data])
filtered_overlap_data

,isbn,user_id,book_rating,book_title
0,051512317X,114368,0,Rising Tides
1,155166416X,114368,5,Sweet Baby
2,1551666618,114368,0,Impetuous Innocent
3,1551660598,114368,10,Without A Trace
4,082177512X,114368,5,Say No to Joe?
...,...,...,...,...
183298,039480001X,11676,10,NaN
183451,030702153X,153662,9,NaN
183454,006440031X,11676,7,NaN
183566,044077456X,11676,10,NaN


In [131]:
filtered_overlap_data['book_rating'] = pd.to_numeric(filtered_overlap_data['book_rating'])

In [132]:
filtered_overlap_data['user_id'].unique()

array(['114368', '153662', '11676', '129358', '69697'], dtype=object)

In [133]:
# create a user_index for each user_id
# category function creates for each user_id a category and then we can use cat.codes to get the index of the user

filtered_overlap_data['user_index'] = filtered_overlap_data['user_id'].astype('category').cat.codes
filtered_overlap_data.iloc[0]

isbn             051512317X
user_id              114368
book_rating               0
book_title     Rising Tides
user_index                0
Name: 0, dtype: object

In [134]:
len(filtered_overlap_data['user_index'].unique())

5

In [135]:
# do the same thing with the isbn
filtered_overlap_data['isbn_index'] = filtered_overlap_data['isbn'].astype('category').cat.codes
filtered_overlap_data.iloc[0]

isbn             051512317X
user_id              114368
book_rating               0
book_title     Rising Tides
user_index                0
isbn_index              779
Name: 0, dtype: object

In [136]:
len(filtered_overlap_data['isbn_index'].unique())

3802

In [137]:
# creating a sparse matrix (doesn't take any memory if there is no rating for a book)

from scipy.sparse import coo_matrix

ratings_coo_mat = coo_matrix((filtered_overlap_data['book_rating'], (filtered_overlap_data['user_index'], filtered_overlap_data['isbn_index'])))

In [138]:
ratings_coo_mat

<5x3802 sparse matrix of type '<class 'numpy.int64'>'
	with 4220 stored elements in COOrdinate format>

In [139]:
print(ratings_coo_mat.row)      # Array of row indices of non-zero values
print(ratings_coo_mat.col)      # Array of column indices of non-zero values
print(ratings_coo_mat.data)     # Array of non-zero values

[0 0 0 ... 1 1 1]
[ 779 1445 1551 ...   77  694 2422]
[ 0  5  0 ...  7 10 10]


In [140]:
ratings_mat = ratings_coo_mat.tocsr()

In [141]:
filtered_overlap_data[filtered_overlap_data['user_id'] == '114368']

,isbn,user_id,book_rating,book_title,user_index,isbn_index
0,051512317X,114368,0,Rising Tides,0,779
1,155166416X,114368,5,Sweet Baby,0,1445
2,1551666618,114368,0,Impetuous Innocent,0,1551
3,1551660598,114368,10,Without A Trace,0,1402
4,082177512X,114368,5,Say No to Joe?,0,1223
...,...,...,...,...,...,...
95,067164257X,114368,5,GARDEN OF SHADOWS (Dollanger Saga (Paperback)),0,975
96,1551668947,114368,5,White Mountain,0,1671
97,037379035X,114368,0,"BODY CONTACT (Blaze, 31)",0,420
98,1551666189,114368,5,Manhunting (Mira),0,1536


In [142]:
# setting the chosen user_index to 'my_index'
my_index = 0

In [143]:
# use cosine similarity to see how similar each user is to us

from sklearn.metrics.pairwise import cosine_similarity

similarity = cosine_similarity(ratings_mat[my_index, :], ratings_mat).flatten()

In [144]:
# how similar are we to ourself?

similarity[my_index]

0.9999999999999981

In [145]:
#how similar are we to the other users?
similarity[2]

0.05096051011181266

In [146]:
# find indices (= position) for 4 users most similar to us
import numpy as np

indices = np.argpartition(similarity, -4)[-4:]

In [147]:
indices

array([1, 0, 3, 4])

In [148]:
# find the user_id for the 4 most similar users

similar_users = filtered_overlap_data[filtered_overlap_data['user_index'].isin(indices)].copy()

In [149]:
#takes ourself out of the list
similar_users = similar_users[similar_users['user_id'] != '114368']


In [150]:
book_recs = similar_users.groupby('isbn').book_rating.agg(['count', 'mean'])

In [151]:
book_recs  

,count,mean
isbn,,
000225929X,1,10.0
000617891X,1,0.0
000648302X,1,8.0
000649840X,1,8.0
002411510X,1,8.0
...,...,...
9879397274,1,7.0
987960170X,1,5.0
9974643058,1,7.0


In [152]:
# include book_titles in book_recs
# Drop duplicates in the data DataFrame based on the 'isbn' column
unique_isbn = average_rating[['isbn', 'book_title', 'rating_count', 'mod_titles']].drop_duplicates(subset='isbn')

# Merge book_recs with the unique_data to add 'book_title' column
book_recs = book_recs.merge(unique_isbn, on='isbn', how='inner')

In [153]:
book_recs

,isbn,count,mean,book_title,rating_count,mod_titles
0,000225929X,1,10.0,MARBLE HEART,4,MARBLE HEART
1,000617891X,1,0.0,At the Stroke of Twelve,2,At the Stroke of Twelve
2,000648302X,1,8.0,Before and After,14,Before and After
3,000649840X,1,8.0,Angelas Ashes,89,Angelas Ashes
4,002411510X,1,8.0,Walden Two (Trade Book),13,Walden Two Trade Book
...,...,...,...,...,...,...
3592,9879397274,1,7.0,Desde Mi Cielo,6,Desde Mi Cielo
3593,987960170X,1,5.0,Kolon,2,Kolon
3594,9974643058,1,7.0,La piel dura,2,La piel dura
3595,9997511417,1,0.0,A Bundle for the Toff,2,A Bundle for the Toff


### Ranking book rocemmendations ###

#### create adjusted book count, e.g. normalized for the count books appeared among people like us vs the rest of people ####

In [154]:
# Find books that are specifically tailored to our taste
book_recs['adjusted_count'] = book_recs['count'] * (book_recs['count'] / book_recs['rating_count'])

In [155]:
# how many times was the book recommended by users like us?
book_recs['score'] = book_recs['mean'] * book_recs['adjusted_count']

In [156]:
book_recs.head()

,isbn,count,mean,book_title,rating_count,mod_titles,adjusted_count,score
0,000225929X,1,10.0,MARBLE HEART,4,MARBLE HEART,0.250000,2.500000
1,000617891X,1,0.0,At the Stroke of Twelve,2,At the Stroke of Twelve,0.500000,0.000000
2,000648302X,1,8.0,Before and After,14,Before and After,0.071429,0.571429
3,000649840X,1,8.0,Angelas Ashes,89,Angelas Ashes,0.011236,0.089888
4,002411510X,1,8.0,Walden Two (Trade Book),13,Walden Two Trade Book,0.076923,0.615385


In [157]:
# take out books that we have already read
book_recs = book_recs[~book_recs['isbn'].isin(unique_books['isbn'])]    

In [158]:
# removing duplicate titles
book_recs = book_recs[~book_recs['book_title'].isin(unique_books['book_title'])]

### Adjusting the amount of similar users and mean in the following 2 cells affects how many recommendations we find:

In [159]:
# at least 1 user similar to us have to have read the book
book_recs = book_recs[book_recs['count'] > 1]

In [162]:
book_recs = book_recs[book_recs['mean'] > 1]

In [163]:
top_10_recs = book_recs.sort_values('score', ascending=False).head(10)  
top_10_recs

,isbn,count,mean,book_title,rating_count,mod_titles,adjusted_count,score
938,067981485X,2,5.5,Teenage Mutant Ninja Turtles: Don't do drugs :...,2,Teenage Mutant Ninja Turtles Don t do drugs ...,2.000000,11.000000
1120,084232044X,2,4.0,Kerry (Grace Livingston Hill),2,Kerry Grace Livingston Hill,2.000000,8.000000
253,037307977X,2,8.5,Heart Of Midnight (Silhouette Intimate Moments...,5,Heart Of Midnight Silhouette Intimate Moments...,0.800000,6.800000
1981,1578565731,2,8.5,Boo,5,Boo,0.800000,6.800000
1952,1576737659,2,4.0,It Had to Be You,3,It Had to Be You,1.333333,5.333333
1708,1562470310,2,5.0,"Happy Birthday, Felicity! A Springtime Story, ...",4,Happy Birthday Felicity A Springtime Story ...,1.000000,5.000000
1983,1578566436,2,4.5,Three Roads Home: Stories of First Love & Seco...,4,Three Roads Home Stories of First Love Seco...,1.000000,4.500000
1607,1558530908,2,5.0,New England Ghosts (American Ghosts Series),5,New England Ghosts American Ghosts Series,0.800000,4.000000
1395,1551667045,2,5.5,Down By The River,6,Down By The River,0.666667,3.666667
1825,1569870225,2,2.5,The Tale of Mr. Jeremy Fisher (Peter Rabbit an...,3,The Tale of Mr Jeremy Fisher Peter Rabbit an...,1.333333,3.333333
